In [1]:
import numpy as np
import _pickle as pickle
import scipy.optimize as optimize
%matplotlib notebook
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import time
import pickle

In [36]:
class NeuralNetwork(object):
    def initialize_weights(self,W1=0,W2=0,b1=0,b2=0):
        if type(W1)== np.ndarray : self.W1 = W1
        else : self.W1 = np.random.randn(self.inputLayerSize,self.hiddenLayerSize)
        if type(W2)== np.ndarray : self.W2 = W2
        else : self.W2 = np.random.randn(self.hiddenLayerSize,self.outputLayerSize)
        if type(b1)== np.ndarray : self.b1 = b1
        else : self.b1= np.random.randn(1,self.hiddenLayerSize)
        if type(b2)== np.ndarray : self.b2 = b2
        else : self.b2= np.random.randn(1,self.outputLayerSize)
            
    def __init__(self, Lambda=0,W1=0,W2=0,b1=0,b2=0):    
        #Define Hyperparameters
        self.inputLayerSize = 12
        self.outputLayerSize = 1
        self.hiddenLayerSize = 25
        
        self.initialize_weights(W1,W2,b1,b2)

        self.Lambda = Lambda 
        
    def forwardPropagation(self, X):
        self.z2 = np.dot(X, self.W1) + self.b1
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.W2) + self.b2
        yHat = self.z3 
        return yHat
        
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def relu(self,z):
        z[z<0] = 0
        return z
    
    def reluPrime(self,z):
        z[z<=0] = 0
        z[z>0] = 1
        return z
    
    def sigmoidPrime(self,z):
        #Gradient of sigmoid
        return np.exp(-z)/((1+np.exp(-z))**2)
    
    def costFunction(self, X, y):
        #Compute cost for given X,y, use weights already stored in class.
        self.yHat = self.forwardPropagation(X)
        j = (0.5*sum((y-self.yHat)**2) + (self.Lambda/2)*(np.sum(self.W1**2)+np.sum(self.W2**2)))/X.shape[0]
        return j
        
    def costFunctionPrime(self, X, y):
        #Compute derivative with respect to W and W2 for a given X and y:
        self.yHat = self.forwardPropagation(X)
        
        #delta3 = np.multiply(-(y-self.yHat), self.reluPrime(self.z3))
        delta3 = -(y-self.yHat)
        #Add gradient of regularization term:
        dJdW2 = (np.dot(self.a2.T, delta3) +  self.Lambda*self.W2)/X.shape[0]
        dJdb2 = np.sum(delta3, axis=0,keepdims=True)/len(delta3)
        
        #print(self.yHat.shape,dJdW2.shape,self.a2.T.shape)
        delta2 = np.dot(delta3, self.W2.T)*self.sigmoidPrime(self.z2)
        #Add gradient of regularization term:
        dJdW1 = (np.dot(X.T, delta2) + self.Lambda*self.W1)/X.shape[0]
        dJdb1 = np.sum(delta2, axis=0,keepdims=True)/len(delta2)
        #print(dJdb1.shape)
        #print('\ndjdw1',dJdW1,'\ndjdw2',dJdW2,'\ndel3',delta3,'\ndel2',delta2)
        return dJdW1, dJdW2,dJdb1,dJdb2
    
    #Helper functions for interacting with other methods/classes
    def getParams(self):
        #Get W1 and W2 Rolled into vector:
        params = np.concatenate((self.W1.ravel(), self.W2.ravel(),self.b1.ravel(),self.b2.ravel()))
        return params
    
    def setParams(self, params):
        #Set W1 and W2 using single parameter vector:
        W1_start = 0
        W1_end = self.hiddenLayerSize*self.inputLayerSize
        self.W1 = np.reshape(params[W1_start:W1_end],(self.inputLayerSize, self.hiddenLayerSize))
        W2_end = W1_end + self.hiddenLayerSize*self.outputLayerSize
        self.W2 = np.reshape(params[W1_end:W2_end],(self.hiddenLayerSize, self.outputLayerSize))
        b1_end = W2_end + self.hiddenLayerSize
        self.b1 = np.reshape(params[W2_end:b1_end],(1,self.hiddenLayerSize))
        b2_end = b1_end + self.outputLayerSize
        self.b2 = np.reshape(params[b1_end:b2_end],(1,self.outputLayerSize))
            
    def computeGradients(self, X, y):
        dJdW1, dJdW2, dJdb1, dJdb2 = self.costFunctionPrime(X, y)
        return np.concatenate((dJdW1.ravel(), dJdW2.ravel(),dJdb1.ravel(),dJdb2.ravel()))
    
class Trainer(object):
    def __init__(self,N,restarts=1):
        self.N=N
        self.restarts=restarts
        
    def callbackF(self,params):
        self.N.setParams(params)
        self.J.append(self.N.costFunction(self.X,self.y))
        self.testJ.append(self.N.costFunction(self.testX,self.testY))
        
    def costFunctionWrapper(self,params,X,y):
        self.N.setParams(params)
        cost = self.N.costFunction(X,y)
        return cost
    
    def train(self,trainX,trainy,testX,testY):
        self.X,self.testX=trainX,testX
        self.y,self.testY=trainy,testY
        
        options = {'maxiter':6000,'disp':True,'gtol':5e-6,'eps':1e-2}
        min_loss=100
        for i in range(self.restarts):
            if i%5==0 : print('res ',i)
            self.J=[]
            self.testJ=[]
            if i!=0 : self.N.initialize_weights()
            params0=self.N.getParams()
            res = optimize.minimize(self.costFunctionWrapper,params0,jac=None,method='L-BFGS-B',args=(trainX,trainy),options=options,callback=self.callbackF)
            if res.fun<min_loss : min_loss,_res,self._J,self._testJ=res.fun,res,self.J,self.testJ
            
        self.N.setParams(_res.x)
        self.optimizationResults = _res

In [6]:
with open("dataa", "rb") as input_file:
   df = pickle.load(input_file)
df['CH2']=[np.nan]*len(df)
for i in range(len(df)):
    if df['CH2_SHELA'].iloc[i]!=-99 :
        if df['CH2_SPIES'].iloc[i]!=-99 :
            df['CH2'].iloc[i]=(df['CH2_SPIES'].iloc[i]+df['CH2_SHELA'].iloc[i])/2
        else :
            df['CH2'].iloc[i]=df['CH2_SHELA'].iloc[i]
    elif df['CH2_SPIES'].iloc[i]!=-99 :
        df['CH2'].iloc[i]=df['CH2_SPIES'].iloc[i]
        df['CH1']=[np.nan]*len(df)
for i in range(len(df)):
    if df['CH1_SHELA'].iloc[i]!=-99 :
        if df['CH1_SPIES'].iloc[i]!=-99 :
            df['CH1'].iloc[i]=(df['CH1_SPIES'].iloc[i]+df['CH1_SHELA'].iloc[i])/2
        else :
            df['CH1'].iloc[i]=df['CH1_SHELA'].iloc[i]
    elif df['CH1_SPIES'].iloc[i]!=-99 :
        df['CH1'].iloc[i]=df['CH1_SPIES'].iloc[i]
    else :
        df['CH1'].iloc[i]=np.nan
dff=df[['K','G','CH1','W1','U','Z','J','W2','I','H','R','CH2','REDSHIFT','REDSHIFT_ERR','ZWARNING']].dropna()
dff[['K','G','CH1','W1','U','Z','J','W2','I','H','R','CH2','REDSHIFT']]=dff[['K','G','CH1','W1','U','Z','J','W2','I','H','R','CH2','REDSHIFT']].replace(-99,np.nan).dropna()
dff=dff.where(dff['REDSHIFT_ERR']<dff['REDSHIFT']*0.1).dropna()
dff=dff.sample(frac=1)
X_data,y_data=dff[['K','G','CH1','W1','U','Z','J','W2','I','H','R','CH2']],dff['REDSHIFT']

C:\Users\Narayan Kundu\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [7]:
inscaler = MinMaxScaler()
Xs_data  = inscaler.fit_transform(X_data)

In [8]:
def modelrun(Lambda=0,restarts=1):
    N=int(len(Xs_data)/4)
    Jtrain,Jtest,Res=[],[],[]
    dz_val=[]
    dz_train=[]
    for i in range(4):
        Xs_train,y_train=list(Xs_data.copy()),list(y_data.copy())
        del Xs_train[i*N:i*N+N]
        del y_train[i*N:i*N+N]
        Xs_val,y_val=Xs_data[i*N:i*N+N],y_data[i*N:i*N+N]
        
        ouscaler= MinMaxScaler()
        ys_train= ouscaler.fit_transform(np.array(y_train).reshape(-1,1))
        ys_val  = ouscaler.transform(np.array(y_val).reshape(-1,1))
        
        Xs_train,Xs_val,ys_train,ys_val,y_val,y_train=np.array(Xs_train),np.array(Xs_val),np.array(ys_train),np.array(ys_val),np.array(y_val),np.array(y_train)
        X,y,testX,testY=Xs_train,ys_train,Xs_val,ys_val
        
        if i>0 : NN=NeuralNetwork(Lambda,W1,W2,b1,b2)
        else : NN=NeuralNetwork(Lambda)
        T=Trainer(NN,restarts)
        T.train(X,y,testX,testY)
        
        Jtrain.append(T._J)
        Jtest.append(T._testJ)
        Res.append(T.optimizationResults)

        pred_train=ouscaler.inverse_transform(NN.forwardPropagation(X)).reshape(len(y_train))
        pred_val=ouscaler.inverse_transform(NN.forwardPropagation(testX)).reshape(len(y_val))
        dz_val.append(np.array((pred_val-y_val)/(1+y_val)))
        dz_train.append(np.array((pred_train-y_train)/(1+y_train)))
        W1,W2,b1,b2=T.N.W1,T.N.W2,T.N.b1,T.N.b2
    print('std train:',np.mean([np.std(dz_train[i]) for i in range(4)]),'\nmean train:',np.mean(np.abs([np.mean(dz_train[i]) for i in range(4)])))
    print('std val:',np.mean([np.std(dz_val[i]) for i in range(4)]),'\nmean val:',np.mean(np.abs([np.mean(dz_val[i]) for i in range(4)])))

    return Jtrain,Jtest,Res,dz_val,dz_train,T
'''start_time=time.time()
Jtrain,Jtest,Res,dz_val,dz_train=modelrun(1e-5,10)
print('evaluation time-',time.time()-start_time)'''

"start_time=time.time()\nJtrain,Jtest,Res,dz_val,dz_train=modelrun(1e-5,10)\nprint('evaluation time-',time.time()-start_time)"

In [27]:
print('std train:',[np.std(dz_train[i]) for i in range(4)],'\nmean train:',[np.mean(dz_train[i]) for i in range(4)])
print('std val:',[np.std(dz_val[i]) for i in range(4)],'\nmean val:',[np.mean(dz_val[i]) for i in range(4)])

std train: [0.2108125249698903, 0.20580485881961674, 0.20205544105120066, 0.20240772345798622] 
mean train: [0.050562191976873144, 0.04023260933867623, 0.03715483680982789, 0.028722106322051074]
std val: [0.1842324277270755, 0.21141350530013647, 0.22496753822566964, 0.20475360795566552] 
mean val: [0.04559936327113747, 0.05569108394099263, 0.023823882170808348, 0.03316345394768969]


In [37]:
start_time=time.time()
Jtrain,Jtest,Res,dz_val,dz_train,T=modelrun(0,15)
#info.append(np.array(modelrun(0,10)))
print('evaluation time-',time.time()-start_time)

res  0
res  5
res  10
res  0
res  5
res  10
res  0
res  5
res  10
res  0
res  5
res  10
std train: 0.23631502150753103 
mean train: 0.036655952350092606
std val: 0.23534554153964327 
mean val: 0.039536084001967405
evaluation time- 2575.0343453884125


In [20]:
with open('DictResDzvalDztrainW1W2b1b2.pkl', 'wb') as f:
   pickle.dump([{'maxiter':6000,'gtol':1e-5,'lambda':0,'res':20},Res,dz_val,dz_train,W1,W2,b1,b2], f)

In [23]:
def modelrun(Lambda=0,restarts=1,W1=0,W2=0,b1=0,b2=0):
    N=int(len(Xs_data)/4)
    Jtrain,Jtest,Res=[],[],[]
    dz_val=[]
    dz_train=[]
    for i in range(4):
        Xs_train,y_train=list(Xs_data.copy()),list(y_data.copy())
        del Xs_train[i*N:i*N+N]
        del y_train[i*N:i*N+N]
        Xs_val,y_val=Xs_data[i*N:i*N+N],y_data[i*N:i*N+N]
        
        ouscaler= MinMaxScaler()
        ys_train= ouscaler.fit_transform(np.array(y_train).reshape(-1,1))
        ys_val  = ouscaler.transform(np.array(y_val).reshape(-1,1))
        
        Xs_train,Xs_val,ys_train,ys_val,y_val,y_train=np.array(Xs_train),np.array(Xs_val),np.array(ys_train),np.array(ys_val),np.array(y_val),np.array(y_train)
        X,y,testX,testY=Xs_train,ys_train,Xs_val,ys_val
        
        NN=NeuralNetwork(Lambda,W1,W2,b1,b2)
        T=Trainer(NN,restarts)
        T.train(X,y,testX,testY)
        
        Jtrain.append(T._J)
        Jtest.append(T._testJ)
        Res.append(T.optimizationResults)

        pred_train=ouscaler.inverse_transform(NN.forwardPropagation(X)).reshape(len(y_train))
        pred_val=ouscaler.inverse_transform(NN.forwardPropagation(testX)).reshape(len(y_val))
        dz_val.append(np.array((pred_val-y_val)/(1+y_val)))
        dz_train.append(np.array((pred_train-y_train)/(1+y_train)))
        W1,W2,b1,b2=T.N.W1,T.N.W2,T.N.b1,T.N.b2
    print('std train:',np.mean([np.std(dz_train[i]) for i in range(4)]),'\nmean train:',np.mean([np.mean(dz_train[i]) for i in range(4)]))
    print('std val:',np.mean([np.std(dz_val[i]) for i in range(4)]),'\nmean val:',np.mean([np.mean(dz_val[i]) for i in range(4)]))

    return Jtrain,Jtest,Res,dz_val,dz_train,W1,W2,b1,b2
start_time=time.time()
Jtrain,Jtest,Res,dz_val,dz_train,W1,W2,b1,b2=modelrun(0,1,W1,W2,b1,b2)
#info.append(np.array(modelrun(0,10)))
print('evaluation time-',time.time()-start_time)

res  0
res  0
res  0
res  0
std train: 0.14385138398537908 
mean train: 0.017771841405767083
std val: 0.15736940545469336 
mean val: 0.018278671305552517
evaluation time- 2098.8036212921143


In [17]:
print('std train:',[np.std(dz_train[i]) for i in range(4)],'\nmean train:',np.abs([np.mean(dz_train[i]) for i in range(4)]))
print('std val:',[np.std(dz_val[i]) for i in range(4)],'\nmean val:',np.abs([np.mean(dz_val[i]) for i in range(4)]))

std train: [0.17258629245730223, 0.16570616599526178, 0.16501018670227346, 0.1612063182866698] 
mean train: [0.02873889 0.0225872  0.02416945 0.02199843]
std val: [0.2430584856418385, 0.20711618431743253, 0.16601741316176905, 0.17271801060344266] 
mean val: [0.03797506 0.04313295 0.02408865 0.00068956]


In [60]:
[(np.std(dz_val[i]),np.mean(dz_val[i])) for i in range(4)]

[(0.31368288370690167, 0.6494939815095458),
 (0.3016873068883493, 0.6578411327809536),
 (0.07685393076630762, -0.14287281458955192),
 (0.31089825650658026, 0.6467057261658093)]